<a href="https://colab.research.google.com/github/nataliepham6720/16-745_Optimal_Control/blob/main/quadrotor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from numpy.linalg import norm, solve
from scipy.spatial.transform import Rotation as R

def hat(v):
    return np.array([
        [0, -v[2], v[1]],
        [v[2], 0, -v[0]],
        [-v[1], v[0], 0]
    ])

def L(q):
    s = q[0]
    v = q[1:4]
    return np.block([
        [np.array([[s]]), -v.reshape(1, 3)],
        [v.reshape(3, 1), s * np.eye(3) + hat(v)]
    ])

def Rmat(q):
    s = q[0]
    v = q[1:4]
    return np.block([
        [np.array([[s]]), -v.reshape(1, 3)],
        [v.reshape(3, 1), s * np.eye(3) - hat(v)]
    ])

T = np.diag([1, -1, -1, -1])
H = np.vstack([np.zeros((1, 3)), np.eye(3)])

def G(q):
    return L(q) @ H

def Q(q):
    return H.T @ (Rmat(q).T @ L(q)) @ H

# Generate a random quaternion
qtrue = np.random.randn(4)
qtrue /= norm(qtrue)
Qtrue = Q(qtrue)

# Generate random world-frame vectors
vN = np.random.randn(3, 10)
vN /= norm(vN, axis=0, keepdims=True)  # Normalize each column
vB = Qtrue.T @ vN  # Body-frame vectors

def residual(q):
    return (vN - Q(q) @ vB).flatten()

# Gauss-Newton Method
q = np.random.randn(4)
q /= norm(q)
phi = np.ones(3)
iter = 0

while np.max(np.abs(phi)) > 1e-8:
    r = residual(q)

    # Compute Jacobian using finite differences (since ForwardDiff is not native to Python)
    def f(q_):
        return residual(q_)

    eps = 1e-8
    dr = np.zeros((len(r), 4))
    for i in range(4):
        dq = np.zeros(4)
        dq[i] = eps
        dr[:, i] = (f(q + dq) - f(q - dq)) / (2 * eps)

    nabla_r = dr @ G(q)
    phi = -solve(nabla_r.T @ nabla_r, nabla_r.T @ r)
    delta_q = np.concatenate([[np.sqrt(1 - phi @ phi)], phi])
    q = L(q) @ delta_q
    q /= norm(q)  # Normalize
    iter += 1

q_est = q
qtrue_est = qtrue

q_est, qtrue_est, iter


(array([ 0.26704275,  0.68633422, -0.22957184,  0.63634132]),
 array([-0.26704275, -0.68633422,  0.22957184, -0.63634132]),
 12)